<a href="https://colab.research.google.com/github/pszachew/Music-Genre-Classification/blob/master/training_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [13]:
X = pd.read_csv('/content/gdrive/MyDrive/csv_data/embedded.csv',header=None)

In [14]:
y = pd.read_csv('/content/gdrive/MyDrive/csv_data/labels.csv', header=None)

,0
0,pop
1,pop
2,pop
3,pop
4,pop
...,...
994,rock
995,rock
996,rock
997,rock
